# Analisi spotify

## Preparazione dei dati per ML

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [4]:
import pandas as pd

# Carica il dataset pulito
df = pd.read_csv('spotify_clean.csv')

# Controllo rapido
print(df.shape)
df.head()

(1500, 14)


,track_name,artist_name,release_date,popularity,danceability,energy,loudness,tempo,instrumentalness,key,mode,release_year,release_month,duration_s
0,Song 0,Artist 0,2015-01-01,61.01005,0.328912,0.951039,-11.654537,106.63049,0.055283,8,0,2015,1,265.503
1,Song 1,Artist 1,2025-12-31,61.01005,0.516030,0.789873,-5.013845,181.61456,0.002502,11,0,2025,12,189.023
2,Song 2,Artist 2,2015-01-01,61.01005,0.874080,0.462060,-3.434210,95.62363,0.060388,8,1,2015,1,189.941
3,Song 3,Artist 3,2015-01-01,61.01005,0.600241,0.641216,-1.451914,183.15892,0.014532,2,1,2015,1,271.677
4,Song 4,Artist 4,2015-01-01,61.01005,0.559638,0.837364,-4.470684,121.04664,0.079854,3,1,2015,1,226.054


In [5]:
# Selezione delle feature
features_num = ['danceability','energy','loudness','tempo','instrumentalness','duration_s']
features_cat = ['key','mode','genre'] if 'genre' in df.columns else ['key','mode']

X = df[features_num + features_cat]
y = df['popularity']

In [7]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: scaling per numeriche + one-hot per categorical
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_num),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), features_cat)
    ])

In [8]:
df.head()

,track_name,artist_name,release_date,popularity,danceability,energy,loudness,tempo,instrumentalness,key,mode,release_year,release_month,duration_s
0,Song 0,Artist 0,2015-01-01,61.01005,0.328912,0.951039,-11.654537,106.63049,0.055283,8,0,2015,1,265.503
1,Song 1,Artist 1,2025-12-31,61.01005,0.516030,0.789873,-5.013845,181.61456,0.002502,11,0,2025,12,189.023
2,Song 2,Artist 2,2015-01-01,61.01005,0.874080,0.462060,-3.434210,95.62363,0.060388,8,1,2015,1,189.941
3,Song 3,Artist 3,2015-01-01,61.01005,0.600241,0.641216,-1.451914,183.15892,0.014532,2,1,2015,1,271.677
4,Song 4,Artist 4,2015-01-01,61.01005,0.559638,0.837364,-4.470684,121.04664,0.079854,3,1,2015,1,226.054


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

# Carica il dataset pulito
df = pd.read_csv('spotify_clean.csv')
print(df.shape)
df.head()

# Selezione delle feature
features_num = ['danceability','energy','loudness','tempo','instrumentalness','duration_s']
features_cat = ['key','mode','genre'] if 'genre' in df.columns else ['key','mode']

X = df[features_num + features_cat]
y = df['popularity']

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: scaling per numeriche + one-hot per categorical
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_num),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), features_cat)  # qui usiamo sparse=False
    ]
)

# Applica il preprocessing (fit sul train)
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

print(X_train_proc.shape, X_test_proc.shape)


(1500, 14)
(1200, 18) (300, 18)
